<a href="https://colab.research.google.com/github/MHan3n/Psych-Han/blob/main/2022Face_recognitionMNE_Hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 29.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import mne
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)
data_path = '/content/E:/NCU/deco_neu/NeuroDecoding2022'
meg_path = data_path + '/MEG'
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif'
raw = mne.io.read_raw_fif(raw_fname)

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
print(raw)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
event_id = {'Famous/1st': 5, 'Famous/2nd_lag0': 6, 'Famous/2nd_lagN': 7,
           'Scrambled/1st': 17, 'Scrambled/2nd_lag0': 18, 'Scrambled/2nd_lagN': 19,
           'Unfamiliar/1st': 13, 'Unfamiliar/2nd_lag0': 14, 'Unfamiliar/2nd_lagN': 15}  

tmin = -0.1         								# pre stimulis interval (in seconds) #
tmax = 1.0          								# post stimulus interval #

#min:(pre onset 0.1), max: (1 sec after onset.)
# if want to see n1, then 0.5sec is enough, but if want to see else then need longer.

#picks = mne.pick_types(raw.info, eeg= False, stim = False)
#if true then all channel kept. 
#if only want certain kind of then use mag, grad, planar1, planar2…etc. exclude = [] can exclude specific channels.
#btw, pilot run found that 跑完reject發現eeg被丟掉的多，乾脆eeg都丟掉了。

#then do artifact rejection.
baseline = (None, 0)  # tmin ~ 0 , -0.1s-0s   
reject = dict(grad=4000e-13, # T / m (gradiometers)
              mag=5e-12, # T (magnetometers)
              )
epoch_run1 = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=baseline,
                    reject = reject, preload = True)

In [ ]:
epoch_run1

In [ ]:
evoked_1 = epoch_run1['Famous'].average()
evoked_2 = epoch_run1['Scrambled'].average()
evoked_3 = epoch_run1['Unfamiliar'].average()

evoked_1.comment = 'Famous'
evoked_2.comment = 'Scrambled'
evoked_3.comment = 'Unfamiliar'

In [ ]:
tmp = [evoked_1, evoked_2, evoked_3]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-02_proc-sss_meg.fif'

raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
#events = mne.find_events(raw, stim_channel='STI101', min_duration = 10 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
epoch_run2 = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=baseline,
                    reject = reject, preload = True)

print(epoch_run2)
print(epoch_run1)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration = 2 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

In [ ]:
epochs_merge = mne.concatenate_epochs([epoch_run1, epoch_run2])
print(epochs_merge)

In [ ]:
evoked_1m = epochs_merge['Famous'].average()
evoked_2m = epochs_merge['Scrambled'].average()
evoked_3m = epochs_merge['Unfamiliar'].average()

evoked_1m.comment = 'Famous'
evoked_2m.comment = 'Scrambled'
evoked_3m.comment = 'Unfamiliar'

tmp = [evoked_1m, evoked_2m, evoked_3m]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-03_proc-sss_meg.fif'

raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
#events = mne.find_events(raw, stim_channel='STI101', min_duration = 10 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
epoch_run2 = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=baseline,
                    reject = reject, preload = True)
print(epoch_run3)
print(epoch_run2)
print(epoch_run1)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration = 2 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

In [ ]:
epochs_merge = mne.concatenate_epochs([epoch_run1, epoch_run2, epoch_run3])
print(epochs_merge)

In [ ]:
evoked_1m = epochs_merge['Famous'].average()
evoked_2m = epochs_merge['Scrambled'].average()
evoked_3m = epochs_merge['Unfamiliar'].average()

evoked_1m.comment = 'Famous'
evoked_2m.comment = 'Scrambled'
evoked_3m.comment = 'Unfamiliar'

tmp = [evoked_1m, evoked_2m, evoked_3m]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-04_proc-sss_meg.fif'

raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
#events = mne.find_events(raw, stim_channel='STI101', min_duration = 10 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
epoch_run2 = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=baseline,
                    reject = reject, preload = True)
print(epoch_run4)
print(epoch_run3)
print(epoch_run2)
print(epoch_run1)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration = 2 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

In [ ]:
epochs_merge = mne.concatenate_epochs([epoch_run1, epoch_run2, epoch_run3.epoch_run4])
print(epochs_merge)

In [ ]:
evoked_1m = epochs_merge['Famous'].average()
evoked_2m = epochs_merge['Scrambled'].average()
evoked_3m = epochs_merge['Unfamiliar'].average()

evoked_1m.comment = 'Famous'
evoked_2m.comment = 'Scrambled'
evoked_3m.comment = 'Unfamiliar'

tmp = [evoked_1m, evoked_2m, evoked_3m]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-05_proc-sss_meg.fif'

raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
#events = mne.find_events(raw, stim_channel='STI101', min_duration = 10 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
epoch_run2 = mne.Epochs(raw, events, event_id, tmin, tmax,
                    baseline=baseline,
print(epoch_run5)             reject = reject, preload = True)
print(epoch_run4)
print(epoch_run3)
print(epoch_run2)
print(epoch_run1)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration = 2 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

In [ ]:
epochs_merge = mne.concatenate_epochs([epoch_run1, epoch_run2, epoch_run3. epoch_run4. epoch_run5])
print(epochs_merge)

In [ ]:
evoked_1m = epochs_merge['Famous'].average()
evoked_2m = epochs_merge['Scrambled'].average()
evoked_3m = epochs_merge['Unfamiliar'].average()

evoked_1m.comment = 'Famous'
evoked_2m.comment = 'Scrambled'
evoked_3m.comment = 'Unfamiliar'

tmp = [evoked_1m, evoked_2m, evoked_3m]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
raw_fname = meg_path + '/derivatives_meg_derivatives_sub-14_ses-meg_meg_sub-14_ses-meg_task-facerecognition_run-06_proc-sss_meg.fif'

raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw, stim_channel='STI101')  #just checking what stim_channel documenting what.
#events = mne.find_events(raw, stim_channel='STI101', min_duration = 10 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

raw.load_data().pick_types(meg=True, stim=True).filter(0, 100, phase= 'zero-double').resample(500)
epoch_run2 = mne.Epochs(raw, events, event_id, tmin, tmax,
print(epoch_run6)                    baseline=baseline,
print(epoch_run5)             reject = reject, preload = True)
print(epoch_run4)
print(epoch_run3)
print(epoch_run2)
print(epoch_run1)

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', min_duration = 2 / raw.info['sfreq'])  #just checking what stim_channel documenting what.

In [ ]:
epochs_merge = mne.concatenate_epochs([epoch_run1, epoch_run2, epoch_run3. epoch_run4. epoch_run5. epoch_run6])
print(epochs_merge)

In [ ]:
evoked_1m = epochs_merge['Famous'].average()
evoked_2m = epochs_merge['Scrambled'].average()
evoked_3m = epochs_merge['Unfamiliar'].average()

evoked_1m.comment = 'Famous'
evoked_2m.comment = 'Scrambled'
evoked_3m.comment = 'Unfamiliar'

tmp = [evoked_1m, evoked_2m, evoked_3m]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
epochs_merge.save('face-recognition-sub014-epo.fif')

In [ ]:
conds= ('Famous' ,'Scrambled', 'Unfamiliar')
evks = dict(zip(conds, [evoked_1m, evoked_2m, evoked_3m]))
#      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ this is equivalent to:
# {'Famous': evoked_1m, 'Scrambled': evoked_2m,
#  'Unfamiliar': evoked_3m}

In [ ]:
evks['Famous'].plot(picks='mag', spatial_colors=True, gfp=True)

In [ ]:
times = np.linspace(0.05, 0.13, 5)
evks['Famous'].plot_topomap(ch_type='mag', times=times, colorbar=True)

In [ ]:
evoked_1m.plot_sensors(show_names=True)

In [ ]:
mne.viz.plot_compare_evokeds(evks, picks='MEG2521',time_unit='ms')

In [ ]:
print(epochs_merge.get_data().shape)
print(epochs_merge.get_data(picks='meg').shape)

In [ ]:
x1 = epochs_merge['Famous'].get_data(picks='meg')
x2 = epochs_merge['Scrambled'].get_data(picks='meg')

y1 = np.zeros(x1.shape[0])
y2 = np.ones(x2.shape[0])

X = np.concatenate((x1, x2), axis = 0)
y = np.concatenate((y1, y2), axis = 0)

print(x1.shape)
print(x2.shape)
print(X.shape)
print(y.shape)

In [ ]:
clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='linear', gamma='auto')
)

time_decod = SlidingEstimator(
    clf, n_jobs=None, scoring='roc_auc', verbose=True)
# here we use cv=3 just for speed
scores_ = cross_val_multiscore(time_decod, X, y, cv=3, n_jobs=None)

# Mean scores across cross-validation splits
scores_FS = np.mean(scores_, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs_merge.times, scores_FS, label='score, logit')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

In [ ]:
x1 = epochs_merge['Famous'].get_data(picks='meg')
x2 = epochs_merge['Unfamiliar'].get_data(picks='meg')

y1 = np.zeros(x1.shape[0])
y2 = np.ones(x2.shape[0])

X = np.concatenate((x1, x2), axis = 0)
y = np.concatenate((y1, y2), axis = 0)

print(x1.shape)
print(x2.shape)
print(X.shape)
print(y.shape)

In [ ]:
clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='linear', gamma='auto')
)

time_decod = SlidingEstimator(
    clf, n_jobs=None, scoring='roc_auc', verbose=True)
# here we use cv=3 just for speed
scores_ = cross_val_multiscore(time_decod, X, y, cv=3, n_jobs=None)

# Mean scores across cross-validation splits
scores_FU = np.mean(scores_, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs_merge.times, scores_FU, label='score, logit')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

In [ ]:
with open('MVPA_scores_s016_SCV.npy', 'wb') as f:
    np.save(f, scores_FS)
    np.save(f, scores_FU)
    np.save(f, epochs_merge.times)

In [ ]:
with open('MVPA_scores_s016_SCV.npy', 'rb') as f:
    scores_FS_load = np.load(f)
    scores_FU_load = np.load(f)
    times = np.load(f)

# Plot
fig, ax = plt.subplots()
ax.plot(times, scores_FS_load, label='score, Famouse vs Scrambled')
ax.plot(times, scores_FU_load, label='score, Famouse vs Unfamiliar')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')